In [1]:
import pandas as pd
import sqlalchemy
from binance.client import cliente
from binance import BinanceSocketManager

Substitua pela key da Binance
 - https://www.youtube.com/watch?v=dII2--v_3lI

In [ ]:
get_ipython().run_line_magic('run', './Binance_Keys.ipynb')

In [ ]:
client = Client(api_key,api_secret)

In [ ]:
bsm = BinanceSocketManager(client)

In [ ]:
socket = bsm.trade_socket(pair)

In [ ]:
def createframe(msg):
    df = pd.DataFrame([msg])
    df = df.loc[:,['s','E','p']]
    df.columns = ['symbol','Time','Price']
    df.Price = df.Price.astype(float)
    df.Time = pd.to_datetime(df.Time, unit='ms')
    return df

In [ ]:
await socket.__aenter__()
msg = await socket.recv()
frame = createframe(msg)
frame.to_sql(pair, engine, if_exists='append', index=False)
print(frame)

Final do Script

Estratégia

In [ ]:
import sqlalchemy
import pandas as pd
from binance.client import Client

In [ ]:
get_ipython().run_line_magic('run', './Binance_Keys.ipynb')

In [ ]:
client = Client(api_key,api_secret)

In [ ]:
engine = sqlalchemy.create_engine('sqlite:///'stream.db')

In [ ]:
#Trendfollowing
#if the crypto was rising by x % -> Buy
#exit when profit is above 0.15% or loss is crossing -0.15%
#Seguidor de Tendência
#Se a criptomoeda estiver subindo x % -> Compre
#Saia quando o lucro estiver acima de 0.15% ou prejuízo acima -0.15%

In [ ]:
def strategy(entry, lookback, qty, open_position=False):
    while True:
        df = pd.read_sql(pair, engine)
        lookbackperiod = df.iloc[-lookback:]
        cumret = (lookbackperiod.Price.pct_change() +1).cumprod() - 1
        if not open_position:
            if cumret[cumret.last_valid_index()] > entry:
                order = client.create_order(symbol=pair,
                                           side='BUY',
                                           type='MARKET',
                                           quantity=qty)
                print(order)
                open_position = True
                break
    if open_position:
        while True:
            df = pd.read_sql('BTCUSDT', engine)
            sincebuy = df.loc[df.Time > 
                              pd.to_datetime(order['transactTime'],
                                            unit='ms')]
            if len(sincebuy) > 1:
                sincebuyret = (sincebuy.Price.pct_change() +1).cumprod() - 1
                last_entry = sincebuyret[sincebuyret.last_valid_index()]
                if last_entry > 0.0015 or last_entry < -0.0015:
                    order = client.create_order(symbol=pair,
                                           side='SELL',
                                           type='MARKET',
                                           quantity=qty)
                    print(order)
                    break

In [ ]:
strategy(0.001, 60, 0.001)